In [10]:
!pip install mediapipe
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
import cv2
import math
from sklearn.model_selection import train_test_split

In [11]:
labels = ['bhujangasana', 'padmasana', 'savasana', 'tadasana', 'trikonasana', 'vrikshasana']

# Function to load and preprocess data
def load_data():
    base_path = '/kaggle/input/yoga-lstm/kaggle/working/MP_Data'
    
    all_label_data = []

    for label in labels:
        asana_path = os.path.join(base_path, label)
        label_data = []

        for file in os.listdir(asana_path):
            file_path = os.path.join(asana_path, file)
            df = pd.read_csv(file_path)  # Assuming CSV contains the sequence data
            label_data.append(df)

        all_label_data.append(label_data)

    return all_label_data

In [12]:
# Function to preprocess sequences
def preprocess_sequences(sequences):
    sequence_length = 50
    num_features = 6
    processed_sequences = []

    for label_data in sequences:
        padded_sequences = []
        for seq in label_data:
            seq_data = seq.to_numpy()
            if len(seq_data) >= sequence_length:
                padded_seq = seq_data[:sequence_length, :]
            else:
                padded_seq = np.zeros((sequence_length, num_features))
                padded_seq[:len(seq_data), :] = seq_data

            padded_sequences.append(padded_seq)
        processed_sequences.append(padded_sequences)

    return processed_sequences

In [13]:
# Function to prepare data for training
def prepare_data(all_label_data):
    X_data = []
    y_data = []
    
    for i, label_data in enumerate(all_label_data):
        for seq in label_data:
            X_data.append(seq[:, :-1])
            y_data.append(i)

    return np.array(X_data), np.array(y_data)

In [5]:
# Load and preprocess data
all_label_data = load_data()
processed_sequences = preprocess_sequences(all_label_data)

# Prepare data for training
X, y = prepare_data(processed_sequences)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/yoga-lstm/kaggle/working/MP_Data\\bhujangasana'

In [6]:
from keras.utils import to_categorical

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert the input data to the appropriate data type
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
# Ensure y_train and y_test are one-hot encoded for multi-class classification
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

NameError: name 'X' is not defined

In [7]:
X[0].shape

(50, 6)

In [7]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [8]:
# Create a Sequential model
model = Sequential()

# Add LSTM layers with dropout and batch normalization
model.add(LSTM(units=128, return_sequences=True, input_shape=(50, 6)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(units=128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Add a Dense output layer for classification
model.add(Dense(units=6, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 128)           69120     
                                                                 
 dropout (Dropout)           (None, 50, 128)           0         
                                                                 
 batch_normalization (Batch  (None, 50, 128)           512       
 Normalization)                                                  
                                                                 
 lstm_1 (LSTM)               (None, 50, 128)           131584    
                                                                 
 dropout_1 (Dropout)         (None, 50, 128)           0         
                                                                 
 batch_normalization_1 (Bat  (None, 50, 128)           512       
 chNormalization)                                       

In [9]:
model.fit(X_train, y_train_encoded, epochs=100, callbacks=[tb_callback])

NameError: name 'X_train' is not defined

In [15]:
y_pred=model.predict(X_test)
predicted_labels = np.argmax(y_pred, axis=1)
predicted_labels

1/1 [==============================] - 0s 76ms/step


array([4, 0, 3, 4, 2, 0, 5, 2, 2, 0, 2, 4, 3, 5, 4, 2, 1, 2, 5, 3, 0, 4,
       4, 4, 2, 1, 0, 0, 1])

In [16]:
y_test

array([4, 0, 3, 4, 2, 0, 5, 2, 2, 0, 2, 3, 3, 5, 4, 2, 1, 2, 5, 3, 0, 4,
       4, 4, 2, 1, 0, 0, 1])

In [17]:
x=0
for i in range(len(y_test)):
    if y_test[i]==predicted_labels[i]:
        x+=1
print(x/len(y_test))

0.9655172413793104


In [14]:
model.save('lstm10fps.h5')

C:\Users\rathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('lstm10fps.h5')

In [16]:
import cv2
import mediapipe as mp
import numpy as np
import math

In [17]:
def calculateAngle(landmark1, landmark2, landmark3):
    x1, y1, z1 = landmark1
    x2, y2, z2 = landmark2
    x3, y3, z3 = landmark3
    
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    if angle < 0:
        angle += 360
    
    return angle

In [ ]:

def classifyAngles_check(landmarks):
    left_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
    
    # Get the angle between the right shoulder, elbow and wrist points. 
    right_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])   
    
    # Get the angle between the left elbow, shoulder and hip points. 
    left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])

    # Get the angle between the right hip, shoulder and elbow points. 
    right_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])

    # Get the angle between the left hip, knee and ankle points. 
    left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])

    # Get the angle between the right hip, knee and ankle points 
    right_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])

#     print(f"Left Elbow Angle: {left_elbow_angle}, "
#           f"Right Elbow Angle: {right_elbow_angle}, "
#           f"Left Shoulder Angle: {left_shoulder_angle}, "
#           f"Right Shoulder Angle: {right_shoulder_angle}, "
#           f"Left Knee Angle: {left_knee_angle}, "
#           f"Right Knee Angle: {right_knee_angle}")
    
    return [left_elbow_angle,right_elbow_angle,left_shoulder_angle,right_shoulder_angle,left_knee_angle,right_knee_angle]


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

labels = ['bhujangasana', 'padmasana', 'savasana', 'tadasana', 'trikonasana', 'vrikshasana']

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)

# Open the webcam
cap = cv2.VideoCapture(0)

frame_count = 0
frames_data = []
capture_frames = False
prediction = ""

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # If 'c' is pressed, start capturing frames
    key = cv2.waitKey(1) & 0xFF
    if key == ord('c'):
        capture_frames = not capture_frames  # Toggle capturing frames

        # Reset frame count and frames_data when starting to capture frames
        if capture_frames:
            frame_count = 0
            frames_data = []
    
    if capture_frames:
        # Process the frame to detect poses
        results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        if results.pose_landmarks:
            # Draw the detected poses on the frame
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            height, width, _ = frame.shape
            rows = 33
            cols = 3
            landmarks = [[0 for _ in range(cols)] for _ in range(rows)]
            landmark = results.pose_landmarks.landmark

            # Get pose landmarks and store them for prediction
            for i in range(len(landmark)):
                landmarks[i] = ((landmark[i].x * width, landmark[i].y * height, landmark[i].z * width))
            angles = classifyAngles_check(landmarks)

            frames_data.append(angles)
            frame_count += 1

            # If 125 frames captured, perform prediction
            if frame_count == 125:
                # Convert frames_data to numpy array for prediction (shape: (125, 33, 3))
                frames_array = np.array(frames_data)
                frames_array = frames_array.reshape((1, 125, 6))

                # Perform prediction
                output = loaded_model.predict(frames_array)
                predicted_label = np.argmax(output)
                prediction = labels[predicted_label]

                frame_count = 0
                frames_data = []

    # Display prediction on the OpenCV window
    cv2.putText(frame, f"Prediction: {prediction}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Pose Detection', frame)

    if key == ord('q'):
        break

# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()
